In [1]:
import wrds
conn = wrds.Connection(wrds_username='ottoriess')

Enter your WRDS username [ottoh]:ottoriess
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [54]:
import pandas as pd
import numpy as np

In [3]:
######################################## Testing IBES
x = conn.raw_sql(
    """ SELECT surprise.ticker
        FROM ibes.surpsum AS surprise
        WHERE surprise.anndats >= '2001-01-01'::date
        ORDER BY surprise.anndats;
    """)
x

KeyboardInterrupt: 

In [70]:
df = conn.raw_sql(
    """ SELECT
        datadate AS date,
        gvkey,
        loc AS country,
        indfmt AS industry,
        sich AS classification,
        at,
        ppegt,
        invt,
        act,
        chee,
        ivao,
        ivst,
        intan,
        gdwl,
        ceq,
        lt,
        lct,
        dlc,
        capx,
        icapt,
        revt,
        xrd,
        ebit,
        ebitda,
        pi,
        nicon,
        oancf,
        COALESCE(txp, txpfs) as txp, -- =tp
        sstk,
        prstkc,
        ivaeq,
        dvt,
        dv,
        dp,
        dltt,
        pstk,
        dltis,
        dltr,
        dlcch,
        at - che AS oa,
        act - che AS coa,
        lct - dlc AS col,
        (at - dlc - dltt - mib - pstk - ceq) AS ol,
        act - che - lct + dlc AS wc,
        at - act - COALESCE(ivao, 0) AS nca,
        lt - lct - dltt AS ncl,
        at - act - COALESCE(ivaeq, 0) - (lt - lct - dltt) AS ncoa,
        ivst + ivao AS fna,
        dltt + dlc + pstk AS fnl,
        (ivst + ivao) - (dltt + dlc + pstk) AS nfna,
        (nicon - oancf) / NULLIF(ABS(nicon), 0) AS poa,
        (nicon - (- sstk + prstkc + dv + oancf + ivncf + fincf)) / NULLIF(ABS(nicon), 0) AS pta,
        at - act - COALESCE(ivao, 0) + act - che - lct + dlc + ivst + ivao - (dltt + dlc + pstk) AS ta,
        act / NULLIF(lct, 0) AS curr_ratio,
        (act - invt) / NULLIF(lct, 0) AS quick_ratio,
        chee / NULLIF(lct, 0) AS cash_ratio,
        oancf / NULLIF(lct, 0) AS opr_cashflow_ratio,
        capx / NULLIF(oancf, 0) AS capx_cashflow_ratio,
        (dlc + dltt) / NULLIF(ceq, 0) AS debt_equity_ratio,
        (dltt + dlc) / NULLIF(at, 0) debt_asset_ratio,
        at / NULLIF(ceq, 0) AS fin_leverage,
        (dltt + dlc) / NULLIF(ebitda, 0) AS debt_ebitda_ratio,
        (dltt + dlc - chee) / NULLIF(ebitda, 0) AS net_debt_ebitda_ratio,
        ebit / NULLIF(xint, 0) AS ebit_int_coverage_ratio,
        ebitda / NULLIF(xint, 0) AS ebitda_int_coverage_ratio,
        ch / NULLIF(xint, 0) AS cash_coverage_ratio,
        (dltt + dlc) / NULLIF(dltt + dlc + teq, 0) AS debt_total_cap_ratio,
        oancf / NULLIF(dlc + dltt, 0) AS cashflow_debt_ratio,
        cogs / NULLIF(invt, 0) AS invt_turnover,
        cogs / NULLIF(ap, 0) AS accounts_payable_turnover,
        revt / NULLIF(COALESCE(rect, artfs), 0) AS rec_turnover,
        365 * invt / NULLIF(cogs, 0) AS days_sales_invt,
        365 * ap / NULLIF(cogs, 0) AS days_sales_payable,
        365 * NULLIF(COALESCE(rect, artfs), 0) / NULLIF(revt, 0) AS days_sales_receivable,
        revt / NULLIF(ppent, 0) AS fixed_asset_turnover,
        revt / NULLIF(at, 0) AS total_asset_turnover,
        (revt - cogs) / NULLIF(revt, 0) AS gross_profit_margin,
        ebitda / NULLIF(revt, 0) AS ebitda_margin,
        ebit / NULLIF(revt, 0) AS ebit_margin,
        pi / NULLIF(revt, 0) AS pre_tax_margin,
        nicon / NULLIF(revt, 0) AS net_profit_margin,
        nicon / NULLIF(at, 0) AS roa,
        nicon / NULLIF(ceq, 0) AS roe,
        COALESCE((ebit * (nicon / NULLIF(pi, 0))) / NULLIF(dlc + dltt + teq, 0), ebit * (1 - txt / NULLIF(pi, 0)) / NULLIF(icapt, 0)) AS roic,
        xrd / NULLIF(revt, 0) AS rd_to_sales_ratio
        
        
        
        
        
FROM
    comp_global_daily.g_funda
WHERE 
    exchg = ANY (ARRAY [104, 107, 132, 151, 154, 171, 192, 194, 201, 209, 256, 257, 273, 276, 286])
    AND curcd = 'EUR';

        """)


In [5]:
##### Accruals (Acc)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','act','chee','lct','dlc','txp','dp','at']]

df2.loc[max(df2.index)+1, :] = None

df2['change_act'] = df2['act'] - df2['act'].shift(1)
df2['change_chee'] = df2['chee'] - df2['chee'].shift(1)
df2['change_lct'] = df2['lct'] - df2['lct'].shift(1)
df2['change_dlc'] = df2['dlc'].fillna(0) - df2['dlc'].shift(1).fillna(0)
df2['change_txp'] = df2['txp'] - df2['txp'].shift(1)
df2['change_chee'] = df2['chee'] - df2['chee'].shift(1)
df2['avg_at'] = (df2['at'] + df2['at'].shift(1))/2

acc = ((df2['change_act'] - df2['change_chee']) - (df2['change_lct'] - df2['change_dlc'] - df2['change_txp']) - df2['dp'])/df2['avg_at']
acc

0          NaN
1     0.030740
2    -0.040623
3    -0.055401
4    -0.123233
5    -0.004306
6     0.008315
7    -0.054317
8     0.013350
9    -0.056480
10   -0.150775
11    0.058354
12    0.091667
13   -0.006740
14   -0.279665
15   -0.026405
16   -0.027095
17   -0.008466
18   -0.015259
19   -0.014258
20   -0.054996
21    0.001935
22         NaN
dtype: float64

In [6]:
##### Change in Current Operating Assets (ChCOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','coa','at']]

df2.loc[max(df2.index)+1, :] = None

chcoa = (df2['coa'] - df2['coa'].shift(1))/df2['at'].shift(1)
chcoa

0          NaN
1     0.419283
2    -0.106170
3    -0.035421
4    -0.043586
5     0.066503
6     0.083223
7    -0.003899
8     0.051644
9    -0.075220
10   -0.062457
11    0.281073
12    0.174344
13    0.007821
14   -0.408335
15    0.017160
16   -0.005014
17    0.023852
18    0.025017
19    0.031414
20    0.028522
21    0.031416
22         NaN
dtype: float64

In [7]:
##### Change in Current Operating Liabilities (ChCOL)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','col','at']]

df2.loc[max(df2.index)+1, :] = None

chcol = (df2['col'].fillna(0) - df2['col'].shift(1))/df2['at'].shift(1)
chcol

0          NaN
1     0.274328
2    -0.145178
3    -0.034984
4     0.026938
5     0.025654
6     0.022254
7     0.014742
8     0.001095
9    -0.053762
10    0.039647
11    0.203251
12    0.036174
13   -0.038813
14   -0.165758
15    0.021504
16   -0.010844
17    0.001337
18    0.015626
19    0.019663
20    0.064571
21   -0.144164
22         NaN
dtype: float64

In [8]:
##### Change in Net Non-Cash Working Capital (ChNNCWC)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','wc','at']]

df2.loc[max(df2.index)+1, :] = None

chnncwc = (df2['wc'].fillna(0) - df2['wc'].shift(1))/df2['at'].shift(1)
chnncwc

0          NaN
1     0.144955
2     0.039008
3    -0.000437
4    -0.070523
5     0.040849
6     0.060969
7    -0.018641
8     0.050549
9    -0.021458
10   -0.102104
11    0.077822
12    0.138170
13    0.046634
14   -0.242577
15   -0.004344
16    0.005830
17    0.022515
18    0.009391
19    0.011751
20   -0.036049
21   -0.061535
22         NaN
dtype: float64

In [27]:
##### Change in Net Non-Current Operating Assets (ChNNCOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','ncoa','at']]

df2.loc[max(df2.index)+1, :] = None

chnncoa = (df2['ncoa'].fillna(0) - df2['ncoa'].shift(1))/df2['at'].shift(1)
chnncoa #drop the last row 22th

0          NaN
1     0.212686
2     0.076880
3    -0.078169
4    -0.063012
5     0.100815
6     0.005018
7    -0.048645
8    -0.011485
9    -0.001234
10   -0.033591
11    0.114122
12    0.062909
13    0.002198
14    0.452430
15    0.113682
16    0.102023
17    0.043109
18   -0.224750
19    0.071196
20    0.029506
21    0.021771
22   -0.563720
dtype: float64

In [29]:
##### Change in Non-Current Operating Assets (ChNCOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','nca', "at"]]

df2.loc[max(df2.index)+1, :] = None

chncoa = (df2["nca"]-df2["nca"].shift(1))/df2['at'].shift(1)
chncoa

0          NaN
1     0.206447
2     0.078344
3    -0.079393
4    -0.061957
5     0.099586
6     0.004286
7    -0.043969
8    -0.010811
9     0.001572
10   -0.006211
11    0.089979
12    0.071224
13    0.002961
14    0.441533
15    0.113279
16    0.107868
17    0.044089
18   -0.224883
19    0.070241
20    0.033817
21    0.022578
22         NaN
dtype: float64

In [30]:
##### Change in Non-Current Operating Liabilities (ChNCOL)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at',"ncl"]]

df2.loc[max(df2.index)+1, :] = None

chncol = (df2["ncl"]-df2["ncl"].shift(1))/df2['at'].shift(1)
chncol

0          NaN
1    -0.006239
2     0.001464
3    -0.001224
4     0.001055
5    -0.001229
6    -0.000732
7     0.004677
8     0.000674
9     0.002806
10    0.027380
11   -0.024143
12    0.008315
13    0.000763
14   -0.010897
15   -0.000403
16    0.005844
17    0.000980
18   -0.000133
19   -0.000955
20    0.004311
21    0.000807
22         NaN
dtype: float64

In [35]:
##### Change in Net Financial Assets (ChNFA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','nfna']]

df2.loc[max(df2.index)+1, :] = None

chnfa = (df2["nfna"].fillna(0) - df2["nfna"].shift(1)) / df2['at'].shift(1)

chnfa


0          NaN
1     0.089190
2    -0.108720
3     0.045023
4    -0.087122
5    -0.168847
6     0.048374
7     0.035573
8     0.049939
9     0.039311
10   -0.116368
11    0.060400
12    0.013505
13    0.072547
14    0.040429
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20   -0.012325
21    0.010471
22         NaN
dtype: float64

In [13]:
##### Change in Long-Term Investments (ChLTI)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','ivao']]

df2.loc[max(df2.index)+1, :] = None

chlti = (df2['ivao'].fillna(0) - df2['ivao'].shift(1))/ df2['at'].shift(1)

chlti


0          NaN
1     0.000000
2     0.000000
3     0.026785
4     0.005512
5    -0.035523
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.016473
13    0.000000
14   -0.013338
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.000000
21    0.000000
22         NaN
dtype: float64

In [14]:
##### Change in Common Equity (ChCE)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','ceq']]

df2.loc[max(df2.index)+1, :] = None

chce = (df2['ceq'] - df2['ceq'].shift(1))/ df2['at'].shift(1)

chce


0          NaN
1     0.571176
2     0.016181
3    -0.073135
4    -0.077404
5     0.062151
6    -0.021997
7     0.046613
8     0.050967
9    -0.001423
10   -0.099574
11    0.199872
12    0.204540
13    0.051876
14    0.470404
15    0.156616
16    0.141318
17    0.032504
18   -0.002022
19   -0.169925
20    0.095835
21    0.016584
22         NaN
dtype: float64

In [15]:
##### Change in Financial Liabilities (ChFL)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','dltt','dlc','pstk']]

df2.loc[max(df2.index)+1, :] = None

chfl = (df2['dltt']+df2['dlc'].fillna(0)+df2['pstk'].fillna(0) - (df2['dltt']+df2['dlc'].fillna(0)+df2['pstk'].fillna(0)).shift(1)) / df2['at'].shift(1)

chfl


0          NaN
1    -0.102610
2     0.108720
3    -0.018230
4     0.092631
5     0.133318
6    -0.048375
7    -0.035578
8    -0.049939
9    -0.039311
10    0.116368
11   -0.060400
12    0.002968
13   -0.072547
14   -0.053766
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.012325
21   -0.004474
22         NaN
dtype: float64

In [16]:
##### Growth in Inventory (GriI)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at',"invt"]]

df2.loc[max(df2.index)+1, :] = None

grii = (df2['invt'] - df2['invt'].shift(1)) / ((df2['at']+df2['at'].shift(1))/2)
grii



0          NaN
1     0.134202
2     0.023487
3    -0.028740
4    -0.060402
5     0.015049
6     0.039660
7     0.009346
8     0.010062
9    -0.009706
10   -0.058111
11    0.100601
12    0.087336
13    0.017349
14   -0.195973
15    0.011246
16   -0.005104
17   -0.000551
18    0.014107
19    0.014398
20    0.000678
21   -0.004990
22         NaN
dtype: float64

In [18]:
##### Inventory Change (ICh)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at',"invt"]]

df2.loc[max(df2.index)+1, :] = None

ich = (df2['invt'] - df2['invt'].shift(1)) / df2['at'].shift(1)
ich

0          NaN
1     0.189916
2     0.023171
3    -0.026781
4    -0.061255
5     0.016745
6     0.039383
7     0.009461
8     0.010111
9    -0.009287
10   -0.061286
11    0.122005
12    0.100575
13    0.016896
14   -0.199354
15    0.012246
16   -0.005452
17   -0.000560
18    0.014202
19    0.013309
20    0.000738
21   -0.005053
22         NaN
dtype: float64

In [19]:
##### Inventory Growth (IGr)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date',"invt"]]

df2.loc[max(df2.index)+1, :] = None

icr = (df2['invt'] - df2['invt'].shift(1)) / df2['invt'].shift(1)
icr

0          NaN
1     0.752433
2     0.095882
3    -0.098409
4    -0.215615
5     0.077268
6     0.206710
7     0.040575
8     0.042696
9    -0.037975
10   -0.238012
11    0.689781
12    0.479696
13    0.070973
14   -0.741034
15    0.181837
16   -0.080680
17   -0.010247
18    0.271589
19    0.202851
20    0.007932
21   -0.063457
22         NaN
Name: invt, dtype: float64

In [ ]:
# M/B and Acruals (MBaAC)
# Check Tobek paper, p.57 (IA5)

In [21]:
##### Net Working Capital Changes (NWCCh)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date',"wc","at"]]

df2.loc[max(df2.index)+1, :] = None

nwcch = (df2['wc'].fillna(0) - df2['wc'].shift(1)) / df2['at'].shift(1)
nwcch

0          NaN
1     0.144955
2     0.039008
3    -0.000437
4    -0.070523
5     0.040849
6     0.060969
7    -0.018641
8     0.050549
9    -0.021458
10   -0.102104
11    0.077822
12    0.138170
13    0.046634
14   -0.242577
15   -0.004344
16    0.005830
17    0.022515
18    0.009391
19    0.011751
20   -0.036049
21   -0.061535
22         NaN
dtype: float64

In [23]:
##### Percent Operating Accruals (POA)
# POA already calculated on the damaframe directly through SQL query

0    -3.225786
1    -0.623939
2    -4.294359
3    -1.548858
4    -3.101194
5    -2.115313
6    -2.261084
7    -3.188035
8    -0.731883
9    -6.467094
10   -1.587917
11   -1.348937
12   -0.159613
13   -4.942090
14    0.953850
15    0.196041
16   -0.111523
17    0.325228
18    0.743458
19    0.129336
20   -0.485871
21    0.073733
22         NaN
Name: poa, dtype: float64

In [ ]:
##### Percent Total Accruals (PTA)
# PTA already calculated in the dataframe through SQL query

In [39]:
##### Total Accruals (TA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date',"wc","ncoa","nfna","at"]]

df2.loc[max(df2.index)+1, :] = None

ta = ((df2["ncoa"]+df2["wc"]+df2["nfna"]).fillna(0)-(df2["ncoa"]+df2["wc"]+df2["nfna"]).shift(1)) / df2['at'].shift(1)
ta

0          NaN
1     0.446831
2     0.007168
3    -0.033584
4    -0.220658
5    -0.027183
6     0.114360
7    -0.031713
8     0.089003
9     0.016618
10   -0.252062
11    0.252345
12    0.214585
13    0.121379
14    0.250282
15    0.109338
16    0.107853
17    0.065625
18   -0.215359
19    0.082947
20   -0.018868
21   -0.607215
22         NaN
dtype: float64

In [ ]:
#### Investment Anomalies:

In [41]:
##### Asset Growth (AGr)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["at"]]

df2.loc[max(df2.index)+1, :] = None

agr = df2["at"] / df2["at"].shift(1)
agr

0          NaN
1     1.830296
2     0.973166
3     0.863652
4     1.028233
5     1.225395
6     0.986009
7     1.024608
8     1.009655
9     0.913683
10    1.109276
11    1.425522
12    1.303187
13    0.947722
14    1.034507
15    1.177717
16    1.136318
17    1.034820
18    1.013471
19    0.848783
20    1.177041
21    1.025194
22         NaN
Name: at, dtype: float64

In [51]:
##### Change in Net Operating Assets (ChNOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["at", "oa", "ol"]]

df2.loc[max(df2.index)+1, :] = None

chnoa = (((df2["oa"]-df2["ol"]) / df2["at"].shift(1)) - (((df2["oa"]-df2["ol"]) / df2["at"].shift(1)).shift(1))) / df2["at"].shift(1)
chnoa

### Note: Need here to adjust the NaNs in the original formulas because otherwise "ol" has many NaNs which produce NaNs here too

0          NaN
1          NaN
2    -0.000383
3    -0.000046
4    -0.000056
5     0.000136
6    -0.000061
7    -0.000073
8     0.000032
9    -0.000033
10   -0.000112
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
dtype: float64

In [52]:
##### Changes in PPE and Inventory-to-Assets (ChPPEIA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["ppegt", "invt", "at"]]

df2.loc[max(df2.index)+1, :] = None

chppeia = ((df2["ppegt"] - df2["ppegt"].shift(1)) + (df2["invt"] - df2["invt"].shift(1))) / df2["at"].shift(1)
chppeia

0          NaN
1     0.295246
2     0.120153
3    -0.071579
4    -0.131630
5     0.051657
6     0.124825
7    -0.025348
8     0.013400
9     0.047731
10   -0.086335
11    0.272187
12    0.167705
13    0.042610
14   -0.527931
15    0.028705
16    0.016487
17    0.011829
18    0.018146
19    0.042612
20    0.042761
21         NaN
22         NaN
dtype: float64

In [55]:
##### Composite Debt Issuance (CDI)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["dltt", "dlc"]]

df2.loc[max(df2.index)+1, :] = None

cdi = np.log((df2["dltt"]+df2["dlc"])/(df2["dltt"].shift(5)+df2["dlc"].shift(5)))
cdi


C:\Users\ottoh\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5     0.907984
6     1.216221
7     0.351014
8     0.239946
9    -0.300838
10   -0.200498
11   -0.294147
12   -0.156441
13   -0.840982
14        -inf
15        -inf
16        -inf
17        -inf
18        -inf
19         NaN
20         inf
21         NaN
22         NaN
dtype: float64

In [56]:
##### Growth in LTNOA (GriLTNOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','act','chee','lct','dlc','txp','dp','at', "oa","ol"]]

df2.loc[max(df2.index)+1, :] = None

df2['change_act'] = df2['act'] - df2['act'].shift(1)
df2['change_chee'] = df2['chee'] - df2['chee'].shift(1)
df2['change_lct'] = df2['lct'] - df2['lct'].shift(1)
df2['change_dlc'] = df2['dlc'].fillna(0) - df2['dlc'].shift(1).fillna(0)
df2['change_txp'] = df2['txp'] - df2['txp'].shift(1)
df2['change_chee'] = df2['chee'] - df2['chee'].shift(1)
df2['avg_at'] = (df2['at'] + df2['at'].shift(1))/2

# Accruals part taken from the acc on top

griltnoa = ((df2["oa"]-df2["ol"]) / df2["at"].shift(1)) - (((df2["oa"]-df2["ol"]) / df2["at"].shift(1)).shift(1)) - ((df2['change_act'] - df2['change_chee']) - (df2['change_lct'] - df2['change_dlc'] - df2['change_txp']) - df2['dp'])/df2['avg_at']

griltnoa

0          NaN
1          NaN
2    -0.257306
3     0.020518
4     0.086681
5     0.095542
6    -0.058960
7    -0.004690
8     0.013119
9     0.028520
10    0.064484
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
dtype: float64

In [57]:
##### Investment (INV)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["revt", "capx"]]

df2.loc[max(df2.index)+1, :] = None

inv = (df2["capx"]/df2["revt"]) / (1/3*((df2["capx"].shift(1)/df2["revt"].shift(1)) + (df2["capx"].shift(2)/df2["revt"].shift(2)) + (df2["capx"].shift(3)/df2["revt"].shift(3))))

inv

0          NaN
1          NaN
2          NaN
3     0.692402
4     0.578771
5     0.950420
6     0.955841
7     0.707647
8     0.808340
9     0.810622
10    0.473614
11    2.234965
12    1.107935
13    0.899434
14    0.447413
15    1.291037
16    1.138674
17    1.005476
18    1.187847
19    1.586460
20    0.609127
21    1.169439
22         NaN
dtype: float64

In [60]:
##### Net Debt Finance (NDF)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["dltis", "dltr", "dlcch", "at"]]

df2.loc[max(df2.index)+1, :] = None

ndf = (df2["dltis"] - df2["dltr"] + df2["dlcch"]) / (1/2*(df2["at"]+df2["at"].shift(1)))
ndf

# fix NaN issue

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
dtype: float64

In [61]:
##### Net Equity Finance (NEF)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["sstk", "prstkc", "dv", "at"]]

df2.loc[max(df2.index)+1, :] = None

nef = (df2["sstk"] - df2["prstkc"] - df2["dv"]) / (1/2*(df2["at"] + df2["at"].shift(1)))
nef

0          NaN
1     0.163867
2    -0.031017
3    -0.031668
4    -0.024503
5    -0.042276
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
dtype: float64

In [63]:
##### Net Operating Asset (NOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["oa", "ol", "at"]]

df2.loc[max(df2.index)+1, :] = None

noa = (df2["oa"] - df2["ol"])/df2["at"].shift(1)
noa

0          NaN
1     0.912214
2     0.614285
3     0.579402
4     0.542851
5     0.634087
6     0.583442
7     0.524434
8     0.550903
9     0.522943
10    0.436651
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20         NaN
21         NaN
22         NaN
dtype: float64

In [66]:
##### Noncurrent Operating Assets Changes (NOACh)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["ncoa", "at"]]

df2.loc[max(df2.index)+1, :] = None

noach = (df2["ncoa"] - df2["ncoa"].shift(1)) / df2["at"]
noach

0          NaN
1     0.116203
2     0.079000
3    -0.059496
4    -0.055921
5     0.053283
6     0.005089
7    -0.047477
8    -0.011375
9    -0.001351
10   -0.030340
11    0.080057
12    0.060286
13    0.002830
14   -0.183887
15    0.015439
16    0.047037
17    0.016104
18    0.010272
19    0.051934
20    0.030273
21    0.037180
22         NaN
dtype: float64

In [67]:
##### Total XFIN (TXFIN)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["sstk", "dv", "prstkc", "dltis", "dltr", "at"]]

df2.loc[max(df2.index)+1, :] = None

txfin = (df2["sstk"] - df2["dv"] - df2["prstkc"] + df2["dltis"] - df2["dltr"]) / df2["at"]
txfin
# Fix NaNs

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
dtype: float64

In [ ]:
# Others from original list:

In [68]:
##### Company Size (cs_ta)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["at"]]

df2.loc[max(df2.index)+1, :] = None

cs_ta = (df2["at"])
cs_ta

0      425.035
1      777.940
2      757.065
3      653.841
4      672.301
5      823.834
6      812.308
7      832.297
8      840.333
9      767.798
10     851.700
11    1214.117
12    1582.221
13    1499.506
14    1551.249
15    1826.933
16    2075.977
17    2148.263
18    2177.202
19    1847.972
20    2175.139
21    2229.939
22         NaN
Name: at, dtype: float64

In [71]:
##### Investment Rate (ir)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["icapt", "ebit", "nicon", "pi"]]

df2.loc[max(df2.index)+1, :] = None

ir = (df2["icapt"] - df2["icapt"].shift(1)) / (df2["ebit"] * (df2["nicon"]/df2["pi"]))
ir

0           NaN
1      3.221391
2     18.429507
3      7.934481
4     -0.002983
5      2.178235
6     -0.199533
7      0.150906
8      0.021874
9     -1.164551
10     0.372743
11     1.423758
12     2.291261
13    -4.763225
14   -29.907235
15     4.024162
16     2.450447
17     1.081096
18    -0.045955
19    -3.583246
20     0.606847
21     0.120338
22          NaN
dtype: float64

In [72]:
##### NOPAT growth (nopat_g)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["icapt"]]

df2.loc[max(df2.index)+1, :] = None

nopat_g = (df2["icapt"] - df2["icapt"].shift(1)) / df2["icapt"]
nopat_g

0          NaN
1     0.680652
2     0.206571
3    -0.180295
4     0.001816
5     0.110357
6     0.079054
7     0.009150
8     0.002440
9    -0.037028
10    0.036507
11    0.239702
12    0.293981
13   -0.042810
14    0.265766
15    0.143741
16    0.132510
17    0.033473
18   -0.002160
19   -0.225373
20    0.105141
21    0.017852
22         NaN
Name: icapt, dtype: float64

In [74]:
##### Revenue 3-Y-CAGR (rev_cagr_3)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["revt"]]

df2.loc[max(df2.index)+1, :] = None

rev_cagr_3 = ((df2["revt"] / df2["revt"].shift(3))**(1/3)) - 1
rev_cagr_3

0          NaN
1          NaN
2          NaN
3     0.077691
4    -0.146301
5     0.103655
6     0.118738
7     0.146761
8     0.081932
9     0.009520
10   -0.123569
11    0.085824
12    0.297986
13    0.338958
14   -0.205942
15   -0.306288
16   -0.221287
17   -0.007873
18    0.105627
19    0.069028
20    0.290097
21    0.216685
22         NaN
Name: revt, dtype: float64

In [75]:
##### EBITDA 3-Y-CAGR (ebitda_cagr_3)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[["ebitda"]]

df2.loc[max(df2.index)+1, :] = None

ebitda_cagr_3 = ((df2["ebitda"] / df2["ebitda"].shift(3))**(1/3)) - 1
ebitda_cagr_3

0          NaN
1          NaN
2          NaN
3    -0.188723
4    -0.406692
5     0.231428
6     0.428529
7     0.476705
8     0.133032
9     0.011503
10   -0.254740
11    0.273211
12    0.453147
13    0.283258
14   -0.558429
15   -0.292366
16    0.022594
17    0.563882
18    0.109619
19    0.024100
20    0.528268
21    0.372467
22         NaN
Name: ebitda, dtype: float64